# Batch Confusion Norm Approach

In [ ]:
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
import keras
from keras import models, Sequential
from keras.layers import Input, Dense, Activation, Flatten, Conv2D, Dropout, Multiply
from keras.layers import AveragePooling2D, MaxPooling2D
from keras.models import Model, Sequential
import matplotlib.pyplot as plt
import time
import numpy as np
from keras.applications.resnet import ResNet50
import tensorflow as tf
from keras import losses
import keras.backend as K
import tensorflow as tf
from keras.models import model_from_json

# Load images stored

In [ ]:
def load_images_augmented(directory):
    
    # This function loads the augmented images and the augmented csv file
    df_train = pd.read_csv(directory + 'augmented.csv')
    train_image = []
    for name in df_train['image_id']:
        name = name.lower()
        path = directory + 'images_resized_augmented/' + name + '.jpg'
        img = cv2.imread(path)
        train_image.append(img)
    train_image_array = np.array(train_image)
    
    return  train_image_array, df_train

In [ ]:
directory = 'C:/Users/julen/OneDrive/Escritorio/IA/CS577-Deep-Learning/Project/'
x_train, df_train = load_images_augmented(directory)

In [ ]:
# Normalize
x_train = x_train / 255.0 

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

y_train = df_train[['healthy', 'multiple_diseases', 'rust', 'scab']].to_numpy()

x_train_original, y_train_original = shuffle(x_train, y_train)

x_train, x_val, y_train, y_val = train_test_split(x_train_original, y_train_original, test_size = 0.2, random_state = 2020)

print('Size of x_train: ', x_train.shape)
print('Size of x_val: ', x_val.shape)

# Batch Confusion Norm

In [ ]:
def model_plot(history):

    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'])
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
def print_score(model, x_test, y_test):
    score = model.evaluate(x_test, y_test, verbose = 0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

In [ ]:
def ASSP(output, act_name): # TODO: Check if it is well built. Not like in paper. Maybe different image size?
    ASSP = Conv2D(filters = 2048, kernel_size = (1,1))(output)
    ASSP = Conv2D(filters = 2048, kernel_size = (3,3), dilation_rate = 2)(ASSP)
#     ASSP = Conv2D(filters = 2048, kernel_size = (3,3), dilation_rate = 4)(ASSP)
    # ASSP = Conv2D(filters = 2048, kernel_size = (3,3), dilation_rate = 6)(ASSP)
    # ASSP = Conv2D(filters = 2048, kernel_size = (3,3), dilation_rate = 7)(ASSP)

    ASSP = Activation(act_name)(ASSP)
    
    return ASSP

In [ ]:
def compute_bcn_loss(y_pred):

    
    matrix = K.dot(K.transpose(y_pred), y_pred)
    eigenvalues, eigenvectors = tf.linalg.eigh(matrix)
    bcn_loss = K.sum(eigenvalues) 
    
    print(eigenvalues)
    print(type(eigenvalues))
    print(type(y_pred))
    print('y_pred shape: ', y_pred.shape)
    print('yPred.transpose shape: ', K.transpose(y_pred).shape)
    print('matrix shape: ', matrix.shape)
    print('eigenvalues shape: ', eigenvalues.shape)
    
    print()
    
    
    
    return bcn_loss



def custom_loss(y_true, y_pred):
    
    categorical_crossentropy = losses.categorical_crossentropy(y_true, y_pred)
    
    BCN_loss = compute_bcn_loss(y_pred)
    
    loss_total = categorical_crossentropy + BCN_loss
    
    return loss_total

In [ ]:
input_shape = (224, 224, 3)
model_resnet = ResNet50(include_top = False, weights = 'imagenet', input_shape = input_shape)
inputs = Input(shape = (224, 224, 3))
out = model_resnet(inputs)
features = ASSP(out, 'relu')
attention = ASSP(out, 'sigmoid')
tensor_total = Multiply()([features, attention])
tensor_total_flat = Flatten()(tensor_total)
tensor_total_flat = Dense(256, activation = 'relu')(tensor_total_flat) # TODO: Check if it should be relu or another
tensor_total_flat = Dense(4, activation = 'softmax')(tensor_total_flat)

model = Model(inputs=inputs, outputs= tensor_total_flat)

model.compile(optimizer = 'adam', loss = custom_loss, metrics = ['categorical_accuracy'])

print(model.summary())

In [ ]:
model.fit(x_train[:120], y_train[:120], batch_size = 2, epochs = 3, validation_data = (x_val, y_val))

# Save weights

In [ ]:
def save_model(model, name):
    """ Saves the model as a Json file"""
    # serialize model to JSON
    model_json = model.to_json()
    with open( str(name) + ".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(name + ".h5")
    print("Saved model to disk") 

In [ ]:
save_model(model, name = 'BCN')